# Early game running start

### Source - https://medium.com/@rqaiserr/how-to-convert-pdfs-into-searchable-key-words-with-python-85aab86c544f

In [ ]:
import pandas as pd
import numpy as np
import regex as re
import requests
import nltk
import PyPDF2 
#import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#nltk.download('punkt') # necessary jaxzz to tokenize
# for downloading pdf's

## Step 1: Clean Main dataset and Download State PDF Contracts Remote->Local

In [ ]:
# Import raw dataset from kaggle
data = pd.read_csv('../raw/contracts.csv', encoding='utf-8')
# Change column headers to utilize underlines instead of whitespace.
data.columns = data.columns.str.replace(' ', '_')
# Use list comprehension to make all headers lowercase as well.
data.columns = [x.lower() for x in data.columns]
# Change contract_url header to say just that...
data.rename(columns={'contract_pdf': 'contract_url'}, inplace=True)
# Output first 5 rows (head)
data.head()

In [ ]:
# How many rows? (before filter)
len(data.index)

In [ ]:
# Filter and create a df with only contracts that have an attached url
data['contract_url'].replace('', np.nan, inplace=True)
data.dropna(subset=['contract_url'], inplace=True)
# How many rows? (after filter should  be 45,775)
len(data.index)

In [11]:
# See where were at again...
data.head()

,purchase_order_description,purchase_order_(contract)_number,revision_number,specification_number,contract_type,start_date,end_date,approval_date,department,vendor_name,vendor_id,address_1,address_2,city,state,zip,award_amount,procurement_type,contract_url
50825,FY 04 HOMELESS SERVICES,2155,0,11655,DELEGATE AGENCY,2003-07-01T00:00:00,2005-06-30T00:00:00,2003-06-26T00:00:00,DEPARTMENT OF HUMAN SERVICES,LAKE FRONT SRO CORP.,3970415Z,4946 N SHERIDAN RD,NaN,CHICAGO,IL,60640,204000.0,RFP,http://ecm.cityofchicago.org/eSMARTContracts/s...
50826,Second Half of Cultural Outreach Program 2012,26792,0,110510,DELEGATE AGENCY,2012-07-01T00:00:00,2013-02-15T00:00:00,2012-08-30T00:00:00,DEPARTMENT OF CULTURAL AFFAIRS,CHANGING WORLDS,91371340Z,329 W 18TH STREET EFT ST,NaN,CHICAGO,IL,60616,6000.0,NaN,http://ecm.cityofchicago.org/eSMARTContracts/s...
50827,Open Space - 4228 W. Ogden,29130,0,121371,COMPTROLLER-OTHER,2013-10-16T00:00:00,2014-12-31T00:00:00,2013-11-08T00:00:00,DEPT OF COMMUNITY DEVELOPMENT,OPENLANDS,93410144P,AMERICAN NATL BK #784699,33 N LA SALLE ST,CHICAGO,IL,60690,192384.1,NaN,http://ecm.cityofchicago.org/eSMARTContracts/s...
50828,DUMP TRAILERS,9982,0,35680,VEHICLES/HEAVY EQUIPMENT (CAPITAL),2005-11-01T00:00:00,2008-10-31T00:00:00,2005-11-08T00:00:00,DEPT OF FLEET MGMT,R G SMITH EQUIPMENT CO.,31303247L,622 NORTHWEST HWY,NaN,DES PLAINES,IL,60018,809580.0,BID,http://ecm.cityofchicago.org/eSMARTContracts/s...
50829,"CAB/CHASSIS WITH UTILITY BODY BOOM, 54' BUCKET...",T24436,0,B10705307,Term Agreement,2002-07-01T00:00:00,2004-06-30T00:00:00,2002-06-21T00:00:00,DEPT OF STREETS & SANITATION,MID-AMERICA TRUCK&EQUIP CO INC,33668302P,625 SOUTH RTE 83,NaN,ELMHURST,IL,60126,0.0,NaN,http://ecm.cityofchicago.org/eSMARTContracts/s...


In [ ]:
# Now use regex to filter and change column to have JUST the url itself:
# Just grab group 1
remove_junk = re.compile(r"{'url':\s'(.*)'}", re.IGNORECASE)
data['contract_url'] = data['contract_url'].apply(lambda x: re.search(remove_junk, x).group(1))

In [ ]:
# At this point for each row we'll download the pdf and store the pdf locally.
    
def download_pdf(row):
    url=row['contract_url']
    response = requests.get(url) # We'll grab the response.text (html output of page), grab REAL pdf link and download.
    # Use regex to pull the link out....Let's hope all chicago contracts follow same html format ;)
    pdf_regex = re.compile(r'<iframe src="(.*)"\sname=', re.IGNORECASE) # find link; this regex should be standard on all state pages.
    new_link = re.search(pdf_regex, response.text).group(1) # REAL pdf link
    r = requests.get(new_link, allow_redirects=True, stream=True)
    with open('../raw/chicago_pdfs/' + str(row['specification_number']) + '-' + str(row['vendor_id']) + '.pdf', 'wb') as f:
        f.write(r.content)

# Download pdf for each row using its contract_url
data.apply(download_pdf, axis=1)



# VERIFIED EXTRACTION CODE TO DOWNLOAD PDF FROM STATE SITES (pain in the butt, took me way too long...)

# url = 'http://ecm.cityofchicago.org/eSMARTContracts/service/DPSWebDocumentViewer?sid=ESMART&id={2488393F-CCF9-476E-808A-9FBF3C25E0D6}'
# response = requests.get(url) # We'll grab the response.text (html output of page), grab real pdf link and download.
# # Use regex to pull the link out....Let's hope all chicago contracts follow same html format ;)
# pdf_regex = re.compile(r'<iframe src="(.*)"\sname=', re.IGNORECASE) # find link; this regex should be standard on all state pages.
# new_link = re.search(pdf_regex, response.text).group(1)
# r = requests.get(new_link, allow_redirects=True, stream=True)
# with open('test.pdf', 'wb') as f:
#     f.write(r.content)

In [ ]:
# This  function says for each row we have an associated pddf, extract text from it. (see what happens!)
def pdf_to_text(row): 
    # Grab what the filename should be
    filename = '../raw/chicago_pdfs/' + str(row['specification_number']) + '-' + str(row['vendor_id']) + '.pdf'
        
# For each row take text from downloaded pdf associated file/delete it?
data['text_list'] = data.apply(pdf_to_text, axis=1)

## STEP 2: Local_PDF -> List of strings representing OCR output

In [ ]:
# https://gitlab.gnome.org/World/OpenPaperwork/pyocr
from wand.image import Image
from PIL import Image as PI
import pyocr
import pyocr.builders
import io
import sys
# Get english language for OCR
from pyocr import tesseract as tool
tool = pyocr.get_available_tools()[0] # this didnt work use tesseract
lang = tool.get_available_languages()[1]
req_image = []
final_text = []
# Convert a pdf to jpg for OCR
image_pdf = Image(filename="../raw/chicago_pdfs/small_example.pdf", resolution=300)
image_jpeg = image_pdf.convert('jpeg')
# Append all pdf converted jpgs to a blob object
for img in image_jpeg.sequence:
    img_page = Image(image=img)
    req_image.append(img_page.make_blob('jpeg'))


In [ ]:
# FINALLY run tesseract-OCR over our blob jpg images
for img in req_image: 
    txt = tool.image_to_string(
        PI.open(io.BytesIO(img)),
        lang='eng',#lang
        builder=pyocr.builders.TextBuilder()
    )
    final_text.append(txt)

In [ ]:
final_text

In [ ]:
# First objective: Create column that contains text from pdf->text (tesseract?)
# (Or better option than tesseract for python if exists...)

# Late game

In [ ]:
# Second objective: Make a column utilizing web scraping on 

In [ ]:
# Third objective: Create truth column that deduces whether or not a contract
# was ACCEPTED/DENIED utiilizing regex and looking at column created above.


In [ ]:
# Fourth objective: Create a list of prioritized vendors to imititate based off:
# -How many contracts they've acquired
# -Value of contracts (ie: give higher weight/importance of imitation to
    # high-paying contract obtainers)

In [ ]:
# Once curated dataset created, upload to Kaggle as a kernel and call TIDY_contract_data.csv/xlsx


In [ ]:
# Create a text classifier that finds and verifies a piece ofo text exists in
# all contracts (ie: a standard or substandard that is followed but sometimes  forgotten)

In [ ]:
# OR create a feature bayesian inference to discern acceptance or denial of contract